Webscraping mit Pyhon: https://realpython.com/beautiful-soup-web-scraper-python/#scrape-the-fake-python-job-site

In [1]:
import requests
from bs4 import BeautifulSoup
from dataclasses import dataclass

@dataclass
class Person:
    name: str
    university: str
    department: str

UHH

In [3]:
uhh_url = "https://www2.informatik.uni-hamburg.de/fiona/pers.php"

page = requests.get(uhh_url)
soup = BeautifulSoup(page.content, "html.parser")

# get all <tr> elements (here: rows of the person table)
persons = soup.find_all(name="tr")
print(f"found {len(persons)} <tr> table row elements")

result = []

for person in persons:
    # only <tr> elements describing actual persons have an id attribute
    if not person.has_attr("id"):
        continue

    # all table field values are insde <a> tags
    fields = person.findAll("a")

    name = fields[1].text           # second <a> always contains the name
    department = fields[2].text

    result.append(Person(
        name=name,
        university="Universität Hamburg",
        department=department
    ))

print(result)


found 349 <tr> table row elements
[Person(name='Fares Abawi', university='Universität Hamburg', department='WTM'), Person(name='Rana Abdullah', university='Universität Hamburg', department='SEMS'), Person(name='Daniel Ahlers', university='Universität Hamburg', department='TAMS'), Person(name='Kyra Ahrens', university='Universität Hamburg', department='WTM'), Person(name='Hassan Ali', university='Universität Hamburg', department='WTM'), Person(name='Dr. Philipp Allgeuer', university='Universität Hamburg', department='WTM'), Person(name='Selenia Anastasi', university='Universität Hamburg', department='LT'), Person(name='Jakob Andersen', university='Universität Hamburg', department='MAST'), Person(name='Johanna Ansohn Mcdougall', university='Universität Hamburg', department='SVS'), Person(name='Saba Anwar', university='Universität Hamburg', department='LT'), Person(name='Dr. Oscar Javier Ariza Nunez', university='Universität Hamburg', department='HCI'), Person(name='Anne Awizen', universi

HAW

In [11]:
haw_url = "https://www.haw-hamburg.de/hochschule/technik-und-informatik/departments/informatik/unser-department/beschaeftigte/"

page = requests.get(haw_url)
soup = BeautifulSoup(page.content, "html.parser")

# get all <tr> elements (here: rows of the person table)
rows = soup.find_all("div", class_="row person-tile filter_tile border-top pt-4")
print(f"found {len(persons)} <tr> table row elements")

for row in rows:
    person = row.find("div", class_="col-9")
    name = person.find("b")
    print(name)

found 67 <tr> table row elements
<b>Johann
												Abrams </b>
<b>Christian
												Ahlf </b>
<b>Jakob
												Andersen </b>
<b>Behnisch
												Awis </b>
<b>Prof. Dr.</b>
<b>Prof. Dr.</b>
<b>Ilona
												Blanck </b>
<b>Holger
												Brackelmann </b>
<b>Michael
												Brodersen </b>
<b>Arne
												Busch </b>
<b>Prof. Dr.</b>
<b>Prof. Dr.</b>
<b>Prof. Dr.</b>
<b>Tobias
												De Gasperis </b>
<b>Maximilian
												De Muirier </b>
<b>Uwe
												Doleschel </b>
<b>Dr.</b>
<b>Prof. Dr.</b>
<b>Prof. Dr.</b>
<b>Timo
												Häckel </b>
<b>Prof. Dr.-Ing.</b>
<b>Dr.</b>
<b>Torge
												Hinrichs </b>
<b>Prof. Dr.</b>
<b>Prof. Dr.</b>
<b>Prof. Dr.</b>
<b>Prof. Dr.</b>
<b>Holger
												Klindtworth </b>
<b>Prof. Dr.-Ing.</b>
<b>Prof. Dr.</b>
<b>Prof. Dr.</b>
<b>Prof. Dr.</b>
<b>Prof. Dr.</b>
<b>Dr. rer. nat.</b>
<b>Prof. Dr.</b>
<b>Tim
												Luecke </b>
<b>Philip
												McClenaghan </b>
<b>Prof. Dr.</b>
<b>Britta
											